In [32]:
import json
import os
import time
import datetime
import numpy as np
import gensim
from config import *
import random
import math
import chars2vec
# import re
import tensorflow as tf
import pickle

files = []
data = {}
data_ID = []
data_len = []
data_y = []
charsVec = ['UNK', 't', '—', '*', 'f', '#', '(', 'a', '”', '{', '!', 'm', 's', ':', 'n', 
     'k', 'z', '}', '@', ')', 'h', '/', 'u', 'o', 'x', '“', 'e', 
     'p', 'i', 'b', '&', ' ', "'", '$', 'r', 'l', '.', '`', '_', 'y', 'c', 
     'w', '?', '~', ';', ']', '+', '^', '%', 'v', 'g', 'q', 'j', '[',
     ',', 'd', '-', '"', '’', '‘', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
charsVocab = {char:idx for (idx, char) in enumerate(charsVec)}
# word2vec = gensim.models.KeyedVectors.load_word2vec_format('/home/hadoop/word2vec.model')
# c2vec = chars2vec.load_model('eng_300')
reward_counter = 0
eval_flag = 0

def get_data_ID():
    global data_ID
    return data_ID

def get_data_len():
    global data_len
    return data_len

def get_curtime():
    return time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))


def list_files(data_path):
    global data, files
    fs = os.listdir(data_path)
    for f1 in fs:
        tmp_path = os.path.join(data_path, f1)
        if not os.path.isdir(tmp_path):
            if tmp_path.split('.')[-1] == 'json':
                files.append(tmp_path)
        else:
            list_files(tmp_path)


def str2timestamp(str_time):
    month = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
             'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
             'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    ss = str_time.split(' ')
    m_time = ss[5] + "-" + month[ss[1]] + '-' + ss[2] + ' ' + ss[3]
    d = datetime.datetime.strptime(m_time, "%Y-%m-%d %H:%M:%S")
    t = d.timetuple()
    timeStamp = int(time.mktime(t))
    return timeStamp


def data_process(file_path):
    ret = {}
    ss = file_path.split("/")
    data = json.load(open(file_path, mode="r", encoding="utf-8"))
    # 'Wed Jan 07 11:14:08 +0000 2015'
    # print("SS:", ss)
    ret[ss[6]] = {'label': ss[5], 'text': [data['text'].lower()], 'created_at': [str2timestamp(data['created_at'])]}
    return ret

def transIrregularWord(word):
    if not word:
        return ''
    pattern1 = "[^A-Za-z]*$" #punctuation at the end of sentence
    pattern2 = "^[^A-Za-z@#]*" #punctuation at the start of sentence
    word = re.sub(pattern2, "", re.sub(pattern1, "", word))
    pattern3 = '(.*)http(.?)://(.*)' # url
    pattern4 = '^[0-9]+.?[0-9]+$' # number
    if not word:
        return ''
#     elif word.__contains__('@'):
#         return 'person'
    elif word.__contains__('#'):
        return 'topic'
    elif re.match(r'(.*)http?://(.*)', word, re.M|re.I|re.S):    
        return 'links'
#     elif re.match(pattern4, word, re.M|re.I):
#         return 'number'
    else:
        return  word

def load_data_fast():
    global data, data_ID, data_len, data_y, eval_flag
    with open("data/data_dict.txt", "rb") as handle:
        data = pickle.load(handle)
    data_ID = np.load("data/data_ID.npy").tolist()
    data_len = np.load("data/data_len.npy").tolist()
    data_y = np.load("data/data_y.npy").tolist()
    max_sent = max( map(lambda value: max(map(lambda txt_list: len(txt_list), value['text']) ), list(data.values()) ) )
    print("max_sent:", max_sent, ",  max_seq_len:", max(data_len))
    tf.flags.DEFINE_integer("max_seq_len", max(data_len), "Max length of sequence (default: 300)")
    tf.flags.DEFINE_integer("max_sent_len", max_sent, "Max length of sentence (default: 300)")
    eval_flag = int(len(data_ID) / 4) * 3
    print("{} data loaded".format(len(data)))    

def sortTempList(temp_list):
    time = np.array([item[0] for item in temp_list])
    posts = np.array([item[1] for item in temp_list])
    idxs = time.argsort().tolist()
    rst = [[t, p] for (t, p) in zip(time[idxs], posts[idxs])]
    del time, posts
    return rst

In [33]:
def load_data(data_path):
    # get data files path
    global data, files, data_ID, data_len, eval_flag
    data = {}
    files = []
    data_ID = []
    data_len = []
    list_files(data_path) #load all filepath to files
    max_sent = 0
    # load data to json
    for file in files:
        td = data_process(file) # read out the information from json file, and organized it as {dataID:{'key':val}}
        for key in td.keys(): # use temporary data to organize the final whole data
            if key in data:
                data[key]['text'].append(td[key]['text'][0])
                data[key]['created_at'].append(td[key]['created_at'][0])
            else:
                data[key] = td[key]
    # convert to my data style
    for key, value in data.items():
        temp_list = []
        for i in range(len(data[key]['text'])):
            temp_list.append([data[key]['created_at'][i], data[key]['text'][i]])
        temp_list = sortTempList(temp_list)
        data[key]['text'] = []
        data[key]['created_at'] = []
        ttext = ""
        last = 0
        for i in range(len(temp_list)):
            if temp_list[i][0] - temp_list[0][0] > FLAGS.time_limit * 3600 or len(data[key]['created_at']) >= 100:
                break
            if i % FLAGS.post_fn == 0: # merge the fixed number of texts in a time interval
                if len(ttext) > 0: # if there are data already in ttext, output it as a new instance
                    words = list( filter(lambda s: len(s)>0, [transIrregularWord(word) for word in re.split('([,\n ]+)', ttext.strip() )]) )
                    if len(words) > max_sent:
                        max_sent = len(words)
                    data[key]['text'].append(words)
                    data[key]['created_at'].append(temp_list[i][0])
                ttext = temp_list[i][1]
            else:
                ttext += " " + temp_list[i][1]
            last = i
        # keep the last one
        if len(ttext) > 0:
            words = list( filter(lambda s: len(s)>0, [transIrregularWord(word) for word in re.split('([,\n ]+)', ttext.strip() )]) )
#             words = list(filter(lambda x:x!=' ' and x!='', re.split('([,\n .]+)', ttext.strip()) ))
            if len(words) > max_sent:
                max_sent = len(words)
            data[key]['text'].append(words)
            data[key]['created_at'].append(temp_list[last][0])
    for key in data.keys():
        data_ID.append(key)
    data_ID = random.sample(data_ID, len(data_ID)) #shuffle the data id
    for i in range(len(data_ID)): #pre processing the extra informations
        data_len.append(len(data[data_ID[i]]['text']))
        if data[data_ID[i]]['label'] == "rumours":
            data_y.append([1.0, 0.0])
        else:
            data_y.append([0.0, 1.0])
    print("max_sent:", max_sent, ",  max_seq_len:", max(data_len))
    # tf.flags.DEFINE_integer("max_seq_len", max(data_len), "Max length of sequence (default: 300)")
    # tf.flags.DEFINE_integer("max_sent_len", max_sent, "Max length of sentence (default: 300)")
    eval_flag = int(len(data_ID) / 4) * 3
    print("{} data loaded".format(len(data)))


In [39]:
import keras

def Word2IDs(word):
    rst = [charsVocab.get(char) for char in word]
    for i in range(len(rst)):
        if rst[i] is None:
            rst[i] = 0
            print("Unknown char:", word[i])
            print("Word:", word)
    return rst

def get_df_batch(start, new_data_len=[]):
    m_data_y = np.zeros([FLAGS.batch_size, 2], dtype=np.int32)
    m_data_len = np.zeros([FLAGS.batch_size], dtype=np.int32)
    if len(new_data_len) > 0:
        t_data_len = new_data_len
    else:
        t_data_len = data_len
    mts = start * FLAGS.batch_size
    if mts >= len(data_ID):
        mts = mts % len(data_ID)
    
    list_batch = []
    for i in range(FLAGS.batch_size):
        m_data_y[i] = data_y[mts]
        m_data_len[i] = t_data_len[mts]
        list_seq = []
        for j in range(t_data_len[mts]):
            t_words = data[data_ID[mts]]['text'][j]
            sentence = list(map(lambda word:Word2IDs(word), t_words))
            sent_arr = keras.preprocessing.sequence.pad_sequences(
                                                    sentence, 
                                                    maxlen= 21, 
                                                    dtype='int32', 
                                                    padding='post', 
                                                    truncating='post',
                                                    value=0.0
                        ).tolist()
            list_seq.append(sent_arr)
        list_batch.append(list_seq)
        mts += 1
        if mts >= len(data_ID): # read data looply
            mts = mts % len(data_ID)
    data_x = np.array(list_batch)
    return data_x, m_data_len, m_data_y

In [35]:
def get_rl_batch(ids, seq_states, stop_states, counter_id, start_id, total_data):
    input_y = np.zeros([FLAGS.batch_size, FLAGS.class_num], dtype=np.float32)
    miss_vec = 0
    total_data = len(data_len)
    for i in range(FLAGS.batch_size):
        # seq_states:records the id of a sentence in a sequence
        # stop_states: records whether the sentence is judged by the program
        if stop_states[i] == 1 or seq_states[i] >= data_len[ids[i]]: 
            ids[i] = counter_id + start_id
            seq_states[i] = 0
            try:
                t_words = data[ data_ID[ids[i]] ]['text'][seq_states[i]]
            except:
                print(ids[i], seq_states[i])
                t_words = []
            if len(t_words) != 0:
                
            input_y[i] = data_y[ids[i]]
            counter_id += 1
            counter_id = counter_id % total_data
        else:
            try:
                t_words = data[ data_ID[ids[i]] ]['text'][seq_states[i]]
            except:
                print("ids and seq_states:", ids[i], seq_states[i])
                t_words = []
            if len(t_words) != 0:
                sentence = list(map(lambda word:Word2IDs(word), t_words))
                sent_arr = keras.preprocessing.sequence.pad_sequences(
                                                        sentence, 
                                                        maxlen= 21, 
                                                        dtype='int32', 
                                                        padding='post', 
                                                        truncating='post',
                                                        value=0.0
                            ).tolist()
            input_y[i] = data_y[ids[i]]
        # point to the next sequence
        seq_states[i] += 1

    return input_x, input_y, ids, seq_states, counter_id


In [36]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

DuplicateFlagError: The flag 'f' is defined twice. First from /Users/lumenglong/anaconda3/envs/TF/lib/python3.7/site-packages/ipykernel_launcher.py, Second from /Users/lumenglong/anaconda3/envs/TF/lib/python3.7/site-packages/ipykernel_launcher.py.  Description from first occurrence: kernel

In [37]:
load_data("/Users/lumenglong/pheme-rnr-dataset/")

max_sent: 31 ,  max_seq_len: 101
5802 data loaded


In [38]:
rst = get_df_batch(15)

Unknown char: ç
Word: começo
Unknown char: ı
Word: fransız
Unknown char: ı
Word: basın
Unknown char: ı
Word: ajansına
Unknown char: ö
Word: göre
Unknown char: ş
Word: kişi
Unknown char: í
Word: podría
Unknown char: ä
Word: mädchen
Unknown char: é
Word: l'amérique
Unknown char: ø
Word: utøya
Unknown char: ï
Word: naïvety
Unknown char: ï
Word: naïvety
Unknown char: é
Word: héros


In [18]:
rst[1]

array([13, 14,  2,  1, 12,  3,  7, 21,  5, 13, 27, 15, 18, 17, 56, 26, 20,
        9, 31,  6, 46, 23,  2,  9,  1,  3, 21, 17, 11,  6,  6,  6, 10, 16,
        3,  2, 10,  4,  4, 13, 12, 21, 22, 24, 22, 18, 17,  6, 21,  5],
      dtype=int32)

In [19]:
rst[2]

array([[0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0]], dtype=int32)

In [31]:
len(rst[0])

50